In [1]:
import sys
sys.path.insert(0, "../..")

In [2]:
import torch
from torch import nn
from torch.utils import data
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import numpy as np
from copy import deepcopy

from nn_extrapolation import AcceleratedSGD

In [3]:
torch.cuda.is_available()

True

In [4]:
val_loss_fn = nn.NLLLoss(reduction="sum")

def validation(model, loader):
    ok = 0
    loss_sum = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.cuda()
            y = y.cuda()
            out = model(x)
            loss_sum += val_loss_fn(out, y)
            preds = out.argmax(1)
            ok += (y == preds).sum()
            total += len(y)
    return ok / total, loss_sum / total

def train_epoch(loss_log):
    model.train()
    for x, y in train_loader:
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss_log += list(loss.flatten().cpu().detach().numpy())
        loss.backward()
        optimizer.step()

In [5]:
train_ds = datasets.MNIST("../../../MNIST", download=True, train=True, transform=transforms.ToTensor())
test_ds = datasets.MNIST("../../../MNIST", download=True, train=False, transform=transforms.ToTensor())
valid_size = int(0.2 * len(train_ds))
train_ds, valid_ds = data.random_split(train_ds, [len(train_ds) - valid_size, valid_size])

train_loader = data.DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2)
valid_loader = data.DataLoader(valid_ds, batch_size=64, shuffle=True, num_workers=2)
test_loader = data.DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=2)

In [6]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 512),
    nn.ReLU(),
    nn.Linear(512, 10),
    nn.LogSoftmax(-1),
)
model.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=10, bias=True)
  (4): LogSoftmax(dim=-1)
)

In [7]:
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, momentum=0.5, weight_decay=1e-5, mode="epoch")
loss_fn = nn.NLLLoss()

In [8]:
log_file = open("SGD-2l_momentum.txt", "w")

## Epoch

In [9]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1
Training loss: 2.1066
Validation accuracy: 0.7222, validation loss: 1.8848
Epoch 2
Training loss: 1.6168
Validation accuracy: 0.7880, validation loss: 1.3583
Epoch 3
Training loss: 1.1503
Validation accuracy: 0.8148, validation loss: 0.9900
Epoch 4
Training loss: 0.8709
Validation accuracy: 0.8380, validation loss: 0.7910
Epoch 5
Training loss: 0.7174
Validation accuracy: 0.8503, validation loss: 0.6762
Epoch 6
Training loss: 0.6247
Validation accuracy: 0.8612, validation loss: 0.6029
Epoch 7
Training loss: 0.5633
Validation accuracy: 0.8693, validation loss: 0.5522
Epoch 8
Training loss: 0.5196
Validation accuracy: 0.8746, validation loss: 0.5158
Epoch 9
Training loss: 0.4868
Validation accuracy: 0.8792, validation loss: 0.4870
Epoch 10
Training loss: 0.4614
Validation accuracy: 0.8828, validation loss: 0.4652
Epoch 11
Training loss: 0.4412
Validation accuracy: 0.8855, validation loss: 0.4471
Epoch 12
Training loss: 0.4245
Validation accuracy: 0.8877, validation loss: 0.4324
E

In [10]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9121, device='cuda:0'), tensor(0.3094, device='cuda:0'))
Valid: (tensor(0.9077, device='cuda:0'), tensor(0.3269, device='cuda:0'))


In [11]:
optimizer.param_groups[0]["method"] = "RNA"
optimizer.accelerate()

In [12]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=10, bias=True)
  (4): LogSoftmax(dim=-1)
)

In [13]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9212, device='cuda:0'), tensor(0.2773, device='cuda:0'))
Valid: (tensor(0.9178, device='cuda:0'), tensor(0.2959, device='cuda:0'))


In [14]:
optimizer.param_groups[0]["method"] = "RRE"
optimizer.accelerate()

In [15]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=10, bias=True)
  (4): LogSoftmax(dim=-1)
)

In [16]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9212, device='cuda:0'), tensor(0.2773, device='cuda:0'))
Valid: (tensor(0.9178, device='cuda:0'), tensor(0.2959, device='cuda:0'))


## Epoch average

In [17]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 512),
    nn.ReLU(),
    nn.Linear(512, 10),
    nn.LogSoftmax(-1),
)
model.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=10, bias=True)
  (4): LogSoftmax(dim=-1)
)

In [18]:
log_file = open("SGD-2l_momentum-avg.txt", "w")
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, momentum=0.5, weight_decay=1e-5, mode="epoch_avg")

In [19]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1
Training loss: 2.1194
Validation accuracy: 0.7188, validation loss: 1.9168
Epoch 2
Training loss: 1.6605
Validation accuracy: 0.7731, validation loss: 1.4067
Epoch 3
Training loss: 1.1904
Validation accuracy: 0.8108, validation loss: 1.0217
Epoch 4
Training loss: 0.8949
Validation accuracy: 0.8306, validation loss: 0.8093
Epoch 5
Training loss: 0.7312
Validation accuracy: 0.8458, validation loss: 0.6870
Epoch 6
Training loss: 0.6331
Validation accuracy: 0.8585, validation loss: 0.6099
Epoch 7
Training loss: 0.5685
Validation accuracy: 0.8662, validation loss: 0.5570
Epoch 8
Training loss: 0.5228
Validation accuracy: 0.8722, validation loss: 0.5191
Epoch 9
Training loss: 0.4890
Validation accuracy: 0.8781, validation loss: 0.4893
Epoch 10
Training loss: 0.4628
Validation accuracy: 0.8813, validation loss: 0.4667
Epoch 11
Training loss: 0.4420
Validation accuracy: 0.8850, validation loss: 0.4482
Epoch 12
Training loss: 0.4249
Validation accuracy: 0.8872, validation loss: 0.4330
E

In [20]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9137, device='cuda:0'), tensor(0.3089, device='cuda:0'))
Valid: (tensor(0.9077, device='cuda:0'), tensor(0.3272, device='cuda:0'))


In [21]:
optimizer.param_groups[0]["method"] = "RNA"
optimizer.accelerate()

In [22]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=10, bias=True)
  (4): LogSoftmax(dim=-1)
)

In [23]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9131, device='cuda:0'), tensor(0.3157, device='cuda:0'))
Valid: (tensor(0.9074, device='cuda:0'), tensor(0.3334, device='cuda:0'))


In [24]:
optimizer.param_groups[0]["method"] = "RRE"
optimizer.accelerate()

In [25]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=10, bias=True)
  (4): LogSoftmax(dim=-1)
)

In [26]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9132, device='cuda:0'), tensor(0.3155, device='cuda:0'))
Valid: (tensor(0.9075, device='cuda:0'), tensor(0.3332, device='cuda:0'))
